# Object Representation in Jupyter

<span style="font-size:1rem; font-weight: bold;">Or how I hacked my way to producing pretty reports with CSS.</span>

Prepared by: Marvin Tensuan

## About Me

I am...

- an accountant 🤓
- a Python user since 2016.
- Working since 2018 | Investment Banking

[![](https://iscaper.com/Images/TwitterIcon.png)]() @marvintensuan    [![](http://www.cloudcompare.org/release/github.png)]() marvintensuan      [![](http://www.michaeldesign.ca/html/images/LinkedIn_Logo20px.png)]() Marvin Tensuan

As much as Python is widely regarded as a language for everyone, adoption still has a long way to go.



## Why Jupyter Notebooks?

yada yada yada

yada yada yada

## The \_repr\_html_ method

In [1]:
class HolyGrailReference:
    def _repr_html_(self):
        return """
            <style>
              #hello-world-python {
                background-color: #d3d3d3;
                font-family: Papyrus;
                font-size: 30px;
                
                text-align: center;
              }
            </style>
            <div id="hello-world-python">
            T'is but a scratch!
            </div>
        """
    
HolyGrailReference()

In [2]:
class FlexRow:
    def _repr_html_(self):
        return """
          <style>
            #flex-row-sample {
              display: flex;
              flex-direction: row;
            }
            #flex-row-sample .child {
              margin: auto;
              background-color: #d3d3d3;
            } 
          </style>
          <div id="flex-row-sample">
            <div class="child">This is the first one.</div>
            <div class="child">This is the second one.</div>
          </div>       
        """

FlexRow()

In [3]:
class FlexColumn:
    def _repr_html_(self):
        return """
          <style>
            #flex-col-sample {
              display: flex;
              flex-direction: column;
            }
            #flex-col-sample .child {
              margin: auto;
              background-color: #d3d3d3;
            } 
          </style>
          <div id="flex-col-sample">
            <div class="child">This is the first one.</div>
            <div class="child">This is the second one.</div>
          </div>       
        """

FlexColumn()

## latexify

In [4]:
import latexify

from typing import Callable, Sequence, TypeVar
T = TypeVar("T")

@latexify.expression
def linreg(
    X: T,
    y: Sequence,
    h: Callable[T, Sequence],
) -> Sequence:
    # A basic regression equation. :D
    m = len(y)
    return (1/(2*m)) * (sum(h(X) - y) ** 2)

linreg

## pandas

In [5]:
from inspect import getsource

from rich import print
from pandas import DataFrame
import numpy as np

In [6]:
df_src = getsource(DataFrame._repr_html_)
print(df_src[:528])

def _repr_html_(self) -> str | None:
        """
        Return a html representation for a particular DataFrame.

        Mainly for IPython notebook.
        """
        if self._info_repr():
            buf = StringIO()
            self.info(buf=buf)
            # need to escape the <class>, should be the first line.
            val = buf.getvalue().replace("<", r"&lt;", 1)
            val = val.replace(">", r"&gt;", 1)
            return "<pre>" + val + "</pre>"

        if get_option("display.notebook_repr_html"):

In [7]:
print(df_src[528:])

max_rows = get_option("display.max_rows")
            min_rows = get_option("display.min_rows")
            max_cols = get_option("display.max_columns")
            show_dimensions = get_option("display.show_dimensions")

            formatter = fmt.DataFrameFormatter(
                self,
                columns=None,
                col_space=None,
                na_rep="NaN",
                formatters=None,
                float_format=None,
                sparsify=None,
                justify=None,
                index_names=True,
                header=True,
                index=True,
                bold_rows=True,
                escape=True,
                max_rows=max_rows,
                min_rows=min_rows,
                max_cols=max_cols,
                show_dimensions=show_dimensions,
                decimal=".",
            )
            return fmt.DataFrameRenderer(formatter).to_html(notebook=True)
        else:
            return None

In [8]:
rng = np.random.default_rng(seed=1234)
df = DataFrame(rng.random((4, 4)), columns=['A', 'B', 'C', 'D'])

print(df.to_html()[:167])

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>A</th>
      <th>B</th>
      <th>C</th>
      <th>D</th>

In [9]:
print(df._repr_html_()[:272])

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">

In [10]:
class DisplayTwoDf:
    def __init__(self, df1, df2):
        self.df1 = df1.to_html()
        self.df2 = df2.to_html()
        
    def _repr_html_(self):
        style = """
          <style>
            .two-df {
              display: flex;
              flex-direction: row;'
            }
            
            .two-df > div {
              margin: auto;
            }
          </style>
        """
        
        return f"""{style}
          <div class="two-df">
            <div>{self.df1}</div>
            <div>{self.df2}</div>
          </div>
        """

In [11]:
DisplayTwoDf(df[['A', 'B']], df[['C', 'D']])

,A,B
0,0.976700,0.380196
1,0.319097,0.118091
2,0.964079,0.263650
3,0.863621,0.863758
,C,D
0,0.923246,0.261692
1,0.241766,0.318534
2,0.441006,0.609871
3,0.674881,0.659874


## pandas Styler

In [12]:
print(getsource(DataFrame.style.fget))

@property
    def style(self) -> Styler:
        """
        Returns a Styler object.

        Contains methods for building a styled HTML representation of the DataFrame.

        See Also
        --------
        io.formats.style.Styler : Helps style a DataFrame or Series according to the
            data with HTML and CSS.
        """
        from pandas.io.formats.style import Styler

        return Styler(self)

In [13]:
style = df.copy().style
style

,A,B,C,D
0,0.976700,0.380196,0.923246,0.261692
1,0.319097,0.118091,0.241766,0.318534
2,0.964079,0.263650,0.441006,0.609871
3,0.863621,0.863758,0.674881,0.659874


In [14]:
print(style.to_html()[:500])

<style type="text/css">
</style>
<table id="T_525b0">
  <thead>
    <tr>
      <th class="blank level0" >&nbsp;</th>
      <th id="T_525b0_level0_col0" class="col_heading level0 col0" >A</th>
      <th id="T_525b0_level0_col1" class="col_heading level0 col1" >B</th>
      <th id="T_525b0_level0_col2" class="col_heading level0 col2" >C</th>
      <th id="T_525b0_level0_col3" class="col_heading level0 col3" >D</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th id="T_525b0_level0_row0" class="r

In [15]:
style = df.copy().style
style.background_gradient(cmap='Blues')

,A,B,C,D
0,0.976700,0.380196,0.923246,0.261692
1,0.319097,0.118091,0.241766,0.318534
2,0.964079,0.263650,0.441006,0.609871
3,0.863621,0.863758,0.674881,0.659874


In [16]:
import seaborn as sns

style = df.copy().style
style.background_gradient(cmap=sns.color_palette('crest', as_cmap=True))

,A,B,C,D
0,0.976700,0.380196,0.923246,0.261692
1,0.319097,0.118091,0.241766,0.318534
2,0.964079,0.263650,0.441006,0.609871
3,0.863621,0.863758,0.674881,0.659874


In [17]:
sns.color_palette('crest')

[(0.48942421, 0.72854938, 0.56751036),
 (0.34892097, 0.64828676, 0.56513633),
 (0.24929311, 0.56486397, 0.5586654),
 (0.15615526, 0.47913212, 0.54850526),
 (0.11131735, 0.39155635, 0.53422678),
 (0.14573579, 0.29354139, 0.49847009)]

**Styler apply functions**

In [18]:
def highlight_third(x):
    return ["background-color: #A0E7E5;",
            "background-color: #A0E7E5;",
            "background-color: #B4F8C8;",
            "background-color: #A0E7E5;"
           ]


style = df.copy().style
print(style.apply.__doc__[:75].strip())
style.apply(highlight_third, axis=1)

Apply a CSS-styling function column-wise, row-wise, or table-wise.

,A,B,C,D
0,0.976700,0.380196,0.923246,0.261692
1,0.319097,0.118091,0.241766,0.318534
2,0.964079,0.263650,0.441006,0.609871
3,0.863621,0.863758,0.674881,0.659874


In [19]:
style = df.copy().style
style.apply(highlight_third, axis=0, subset=['A', 'B'])

,A,B,C,D
0,0.976700,0.380196,0.923246,0.261692
1,0.319097,0.118091,0.241766,0.318534
2,0.964079,0.263650,0.441006,0.609871
3,0.863621,0.863758,0.674881,0.659874


In [20]:
style = df.copy().style
print(style.applymap.__doc__[:50].strip())

style.applymap(
    lambda value: f"background-color: #a0e7e5;"
    if value < 0.5 else None
)

Apply a CSS-styling function elementwise.

,A,B,C,D
0,0.976700,0.380196,0.923246,0.261692
1,0.319097,0.118091,0.241766,0.318534
2,0.964079,0.263650,0.441006,0.609871
3,0.863621,0.863758,0.674881,0.659874


In [21]:
style = df.copy().style
print(style.apply_index.__doc__[:75].strip())

s = "background-color: #254b7f; color: white;"
style.apply_index(lambda _: [s] * 4, axis=1)

Apply a CSS-styling function to the index or column headers, level-wise.

,A,B,C,D
0,0.976700,0.380196,0.923246,0.261692
1,0.319097,0.118091,0.241766,0.318534
2,0.964079,0.263650,0.441006,0.609871
3,0.863621,0.863758,0.674881,0.659874


In [22]:
style = df.copy().style
print(style.applymap_index.__doc__[:75].strip())

bg_color = '#254b7f'
font_color = 'white'
style.applymap_index(
    lambda value, bg, font: f"background-color: {bg}; color: {font};" if value % 2 == 0 else None,
    bg=bg_color, font=font_color)

Apply a CSS-styling function to the index or column headers, elementwise.

,A,B,C,D
0,0.976700,0.380196,0.923246,0.261692
1,0.319097,0.118091,0.241766,0.318534
2,0.964079,0.263650,0.441006,0.609871
3,0.863621,0.863758,0.674881,0.659874


## What to Do From Here?

### nbconvert

nbconvert is a tool that converts Jupyter Notebook into various other formats, such as HTML, PDF, mardown and Python scripts.

### papermill

papermill is a Python library that parameterizes Jupyter notebooks.

## Summary

- Jupyter Notebook/Lab provides a powerful platform to interact and experiment with data. Being a web application, it can harness the inherent power that a web browser has.
- CSS tells the browser how an object should look like.
- The `_repr_html_` method defines how an object is represented. Libraries are taking advantage of this fact to improve DX.

## Learn More!

- Look up: HTML5, semantic HTML
- CSS Frameworks, ~~[State of CSS](https://stateofcss.com/en-us/)~~
- Mozilla MDN docs
- pandas Official Docs Guide on [Table Visualization](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)

# &#35;

## Appendix

The DOM tree

📷 Wikipedia

[![](https://upload.wikimedia.org/wikipedia/commons/5/5a/DOM-model.svg)]()